In [1]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']

In [5]:
region = sess.boto_session.region_name
print(region)

ap-northeast-1


In [6]:
# define docker image url
image_name = 'bdai-test-docker-repo'
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, image_name)
print(image)

360114749021.dkr.ecr.ap-northeast-1.amazonaws.com/bdai-test-docker-repo:latest


In [7]:
bucket = 'bdai-sagemaker-test'
prefix = 'sagemaker/xgboost-tut/data'
output_location  = 's3://{}/{}/output'.format(bucket, prefix)
print(output_location)

s3://bdai-sagemaker-test/sagemaker/xgboost-tut/data/output


In [8]:
hyperparameters = {
    "max_depth": 10
}

In [17]:
objective_metric_name = 'validation-accuracy'
objective_metrics = [
    {
        "Name": "validation-accuracy",
        "Regex": "validation-accuracy=([0-9\\.]+)"
    }
]

In [14]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {
    'max_depth': IntegerParameter(1, 10)
}

In [9]:
tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.m4.xlarge',
                       output_path=output_location,
                       sagemaker_session=sess,
                       hyperparameters=hyperparameters)

In [18]:
tunner = HyperparameterTuner(
    tree,
    objective_metric_name,
    hyperparameter_ranges,
    objective_metrics,
    max_jobs=9
)

In [10]:
training_data_location = 's3://{}/{}/input/training'.format(bucket, prefix)
testing_data_location = 's3://{}/{}/input/testing'.format(bucket, prefix)
print(training_data_location)
print(testing_data_location)

s3://bdai-sagemaker-test/sagemaker/xgboost-tut/data/input/training
s3://bdai-sagemaker-test/sagemaker/xgboost-tut/data/input/testing


In [19]:
tunner.fit({'training': training_data_location, 'testing': testing_data_location})

INFO:sagemaker:Creating hyperparameter tuning job with name: bdai-test-docker-rep-180928-1037


In [11]:
tree.fit({'training': training_data_location, 'testing': testing_data_location})

INFO:sagemaker:Creating training-job with name: bdai-test-docker-repo-2018-09-28-09-27-06-604


....................
Starting the training.
{'max_depth': '10'}
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
['setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa' 'setosa'
 'virginica' 'versicolor' 'virginica' 'virginica' 'virginica' 'virginica'
 'virginica' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'virginica' 'versicolor' 'versicolor' 'versicolor' 'versicolor'
 'versicolor']
Training complete.

Billable seconds: 32


In [30]:
tree.hyperparameters()

{'k': 1}

In [12]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

INFO:sagemaker:Creating model with name: bdai-test-docker-repo-2018-09-26-04-33-50-006
INFO:sagemaker:Creating endpoint with name bdai-test-docker-repo-2018-09-26-04-21-09-840


---------------------------------------------------------------!

In [13]:
import boto3
client = boto3.client('sagemaker-runtime')

In [23]:
# custom_attributes = "c000b4f9-df62-4c85-a0bf-7c525f9104a4"  # An example of a trace ID.
endpoint_name = "bdai-test-docker-repo-2018-09-26-04-21-09-840"                                       # Your endpoint name.
content_type = "text/csv"                                        # The MIME type of the input data in the request body.
# accept = ""                                              # The desired MIME type of the inference in the response.
payload = "setosa,5.1,3.5,1.4,0.2"                                             # Payload for inference.

In [24]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Body=payload
    )
response_body = response['Body']
prediction = response_body.read().decode('utf-8')
print(prediction)

setosa



In [31]:
import sagemaker
sagemaker.Session().delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: bdai-test-docker-repo-2018-09-26-04-21-09-840
